In [2]:
import numpy as np

In [6]:
# Løser modellen via backwards induction.
#Denne er den simple med et state to valg. Som virker. Ingen stokastik.

#Angiv parameterværdier. 
SU=0.8 # Det ændrer hvad man vælger
beta=1.0 #Discount factor
B=0.5 #Coefficient til humankapital
T=3 #Antal tidsperioder
max_HK=3 #Maksimalt niveau af uddannelse.


#Initialize:


ev0 = np.nan+np.zeros([max_HK,T]) #Generer expected value function.
pk = np.nan+np.zeros([max_HK,T]) # Choice prob for choosing work
V_school=np.nan+np.zeros([max_HK,T]) # Choice specific value functions
V_work=np.nan+np.zeros([max_HK,T]) # Choice specific value functions
maxV=np.nan+np.zeros([max_HK,T]) # Choice specific value functions

#Now I generate the expected value for THE LAST PERIOD from both alternatives and the value function (which is the max of the two)
V_school[:,T-1]=SU # Last period value from schooling
V_work[:,T-1]=B*np.arange(max_HK) # Last period value from work depending on HK
ev0[:,T-1]=np.maximum(V_school[:,T-1],V_work[:,T-1]) 

#Now I generate the choice probabilities in the last period 
pk[:,T-1]=ev0[:,T-1]==V_work[:,T-1] #Uden stokastik
#pk[:,T-1]=np.exp(V_work[:,T-1])/(np.exp(V_school[:,T-1])+np.exp(V_work[:,T-1])) # MED EV1 error terms er det logit form expression. 


#Her løser jeg modellen via backwards induction for periode T-1 indtil for periode 0
for t in range(T-2, -1, -1): #Solve backwards from period T-1(2??), until period 0, backwards
    for h in range(max_HK):
        V_school[h,t]=SU + beta*ev0[np.minimum(h+1,max_HK-1),t+1] #Denne viser værdien af skole som er, værdien af SU samt den forventede value function hvor vi opdaterer hk., i t+1
        V_work[h,t]= B*h + beta*ev0[h,t+1] #Denne viser værdien af work som er lig state space matricen evalueret der hvor vi er + den forventede value function i periode t+1
        maxV[h,t]=np.maximum(V_work[h,t],V_school[h,t]) #Denne viser expected value function som er lig den maksimale værdi vi får fra den chocie specific value fct.
        #pk[h,t]=ev0[h,t]==V_work[h,t] #Uden stokastik
                # recenter Bellman by subtracting max(VK, VR)
        #maxV = np.maximum(value_keep,value_replace) 
        ev0[h,t] = (maxV[h,t] + np.log(np.exp(V_school[h,t]-maxV[h,t])  +  np.exp(V_work[h,t]-maxV[h,t])))  # This is the Logsum 
        
        pk[h,t]=np.exp(V_work[h,t])/(np.exp(V_school[h,t])+np.exp(V_work[h,t])) # computer choice probabilities som er closed form når det er logit.
        #print(pk)

#print(hk)
print(pk)
#print(V_school)
#print(V_work)
print(ev0)  

[[0.24918264 0.31002552 0.        ]
 [0.3488582  0.37754067 0.        ]
 [0.549834   0.549834   1.        ]]
[[3.36066983 1.97110067 0.8       ]
 [3.82716672 2.27407698 0.8       ]
 [4.19627774 2.59813887 1.        ]]


In [62]:
# Simuler modellen

import pandas as pd
N=10 #Amount of people to simulate. 
idx = np.tile(np.arange(1,N+1),(T,1))  # Index
t = np.tile(np.arange(1,T+1),(N,1)).T # Tidsperiode for hvert individ.

x=np.zeros([T,N]) # states for all the simulated people in all the simulated time. 
x1=np.zeros([T,N]) # states for all the simulated people in all the simulated time. 
d=np.nan + np.zeros((T,N)) # decisions for all people. 1 = work, 0 = school. 
u_init=0
u_d=np.random.rand(T,N) # Den der gør om man vælger den ene eller den anden.
#pks=np.zeros([max_HK,N,T])
#print(pks)
x[0,:]=u_init # First state variable. 


#print(pk[int(x[1,1]),1])

for n in range(N):
    for it in range(T):
        d[it,n] = u_d[it,n]<pk[int(x[it,n]),it] # d=1=work. jo mindre u bliver jo mere ssh for work. Så vel i virkeligheden 
        x1[it,n] = np.minimum(x[it,n]+(1-d[it,n]),max_HK-1)
        if it < T-1:
            x[it+1,n] = x1[it,n]    

idx =  np.reshape(idx,T*N,order='F')
t = np.reshape(t,T*N,order='F')
d = np.reshape(d,T*N,order='F')
x = np.reshape(x,T*N,order='F')
x1 = np.reshape(x1,T*N,order='F')
#dx1 = np.reshape(dx1,T*N,order='F')


data = {'id': idx,'t': t, 'd': d, 'x': x, 'x1': x1}
df= pd.DataFrame(data)
    
df

,id,t,d,x,x1
0,1,1,0.0,0.0,1.0
1,1,2,0.0,1.0,2.0
2,1,3,0.0,2.0,2.0
3,2,1,1.0,0.0,0.0
4,2,2,0.0,0.0,1.0
5,2,3,0.0,1.0,2.0
6,3,1,0.0,0.0,1.0
7,3,2,0.0,1.0,2.0
8,3,3,1.0,2.0,2.0
9,4,1,0.0,0.0,1.0


In [8]:
#Nu er det så et fjerde state. Efter det har vi en rigtig model

#Angiv parameterværdier. 
SU=0.8 # Det ændrer hvad man vælger
beta=1.0 #Discount factor
B=0.5 #Coefficient til humankapital
C=0.35 #Coefficient til erfaring
K_b=1 #Constant til blue collar work
B_b=0.1 #Coefficient til HK, blue collar
C_b=0.2 #Coefficient til egen blue collar erfaring
H=1.0 #Coefficient til at være hjemme.
T=5 #Antal tidsperioder
max_HK=4 #Maksimalt niveau af uddannelse.
max_e=3 #Maksimalt niveau af erfaring. Samme som tidsperioder.
max_eb=2 #Maksimalt niveau af blue collar erfaring. Samme som tidsperioder. 

#Generer min expected value function. Nu skal den have en ekstra dimension.

ev0 = np.nan+np.zeros([max_HK,max_e,max_eb,T]) #Generer expected value function med ekstra dimension (første er hk, anden er erfaring, så kommer blue collar erfaring, sidste er tid)
#print(ev0) #Tid er kolonner, blue_erfaring er rækker, stor matrix er HK, lille matrix erfaring
pk_w = np.nan+np.zeros([max_HK,max_e,max_eb,T]) # Choice prob for choosing work
pk_b = np.nan+np.zeros([max_HK,max_e,max_eb,T]) #Choice prob for choosing blue collar
pk_s = np.nan+np.zeros([max_HK,max_e,max_eb,T]) # Choice prob for choosing school
V_school=np.nan+np.zeros([max_HK,max_e,max_eb,T]) # Choice specific value functions
V_work=np.nan+np.zeros([max_HK,max_e,max_eb,T]) # Choice specific value functions
V_home=np.nan+np.zeros([max_HK,max_e,max_eb,T]) # Choice specific value functions
V_blue=np.nan+np.zeros([max_HK,max_e,max_eb,T]) # Choice specific value functions
maxV=np.nan+np.zeros([max_HK,max_e,max_eb,T]) # Choice specific value functions

#Nu genererer jeg expected value i last period for hver af de enkelte valg:
V_school[:,:,:,T-1]=SU # Last period value from schooling
#print(V_school)

for e in range(max_e):
    for eb in range(max_eb):
        V_work[:,e,eb,T-1]=B*np.arange(max_HK)+C*e + 0*eb

#print(V_work)

for e in range(max_e):
    for eb in range(max_eb):
        V_blue[:,e,eb,T-1]=B_b*np.arange(max_HK)+0*e + C_b*eb

V_home[:,:,:,T-1]=H
#print(V_home)

ev0[:,:,:,T-1]=np.maximum.reduce([V_work[:,:,:,T-1],V_school[:,:,:,T-1],V_home[:,:,:,T-1],V_blue[:,:,:,T-1]])
#print(ev0)

pk_w[:,:,:,T-1]=ev0[:,:,:,T-1]==V_work[:,:,:,T-1] #Uden stokastik
#pk_w[:,:,:,T-1]=np.exp(V_work[:,:,:,T-1])/(np.exp(V_school[:,:,:,T-1])+np.exp(V_work[:,:,:,T-1])+np.exp(V_home[:,:,:,T-1])+np.exp(V_blue[:,:,:,T-1])) # MED EV1 error terms er det logit form expression.
pk_s[:,:,:,T-1]=ev0[:,:,:,T-1]==V_school[:,:,:,T-1] #Uden stokastik
#pk_s[:,:,:,T-1]=np.exp(V_school[:,:,:,T-1])/(np.exp(V_school[:,:,:,T-1])+np.exp(V_work[:,:,:,T-1])+np.exp(V_home[:,:,:,T-1])+np.exp(V_blue[:,:,:,T-1])) # MED EV1 error terms er det logit form expression.
pk_b[:,:,:,T-1]=ev0[:,:,:,T-1]==V_blue[:,:,:,T-1] #Uden stokastik
#pk_b[:,:,:,T-1]=np.exp(V_blue[:,:,:,T-1])/(np.exp(V_school[:,:,:,T-1])+np.exp(V_work[:,:,:,T-1])+np.exp(V_home[:,:,:,T-1])+np.exp(V_blue[:,:,:,T-1])) # MED EV1 error terms er det logit form expression.


for t in range(T-2, -1, -1): #Solve backwards from period T-1(2??), until period 0, backwards
    for h in range(max_HK):
        for ex in range(max_e):
            for e_b in range(max_eb):
                V_school[h,ex,e_b,t]=SU + beta*ev0[np.minimum(h+1,max_HK-1),ex,e_b,t+1] #Denne viser værdien af skole som er, værdien af SU samt den forventede value function hvor vi opdaterer hk., i t+1
                V_work[h,ex,e_b,t]= B*h + C*ex + beta*ev0[h,np.minimum(ex+1,max_e-1),e_b,t+1] #Denne viser værdien af work som er lig state space matricen evalueret der hvor vi er + den forventede value function i periode t+1
                V_home[h,ex,e_b,t]=H + beta*ev0[h,ex,e_b,t+1] #Denne viser værdien af home.
                V_blue[h,ex,e_b,t]=K_b+B_b*h+C_b*e_b+beta*ev0[h,ex,np.minimum(e_b+1,max_eb-1),t+1] # Værdien af blue collar work
                maxV[h,ex,e_b,t]=np.maximum.reduce([V_work[h,ex,e_b,t],V_school[h,ex,e_b,t],V_home[h,ex,e_b,t],V_blue[h,ex,e_b,t]]) #Denne viser expected value function som er lig den maksimale værdi vi får fra den chocie specific value fct.
                #pk_w[h,ex,e_b,t]=ev0[h,ex,e_b,t]==V_work[h,ex,e_b,t] #Uden stokastik
                #pk_s[h,ex,e_b,t]=ev0[h,ex,e_b,t]==V_school[h,ex,e_b,t]
                #pk_b[h,ex,e_b,t]=ev0[h,ex,e_b,t]==V_blue[h,ex,e_b,t]
                
                ev0[h,ex,e_b,t] = (maxV[h,ex,e_b,t] + np.log(np.exp(V_school[h,ex,e_b,t]-maxV[h,ex,e_b,t])  +  np.exp(V_work[h,ex,e_b,t]-maxV[h,ex,e_b,t]) + np.exp(V_home[h,ex,e_b,t]-maxV[h,ex,e_b,t]) + np.exp(V_blue[h,ex,e_b,t]-maxV[h,ex,e_b,t])))
                
                
                pk_w[h,ex,e_b,t]=np.exp(V_work[h,ex,e_b,t])/(np.exp(V_school[h,ex,e_b,t])+np.exp(V_work[h,ex,e_b,t])+np.exp(V_home[h,ex,e_b,t])+np.exp(V_blue[h,ex,e_b,t])) # computer choice probabilities som er closed form når det er logit.
                pk_s[h,ex,e_b,t]=np.exp(V_school[h,ex,e_b,t])/(np.exp(V_school[h,ex,e_b,t])+np.exp(V_work[h,ex,e_b,t])+np.exp(V_home[h,ex,e_b,t])+np.exp(V_blue[h,ex,e_b,t])) # computer choice probabilities som er closed form når det er logit.
                pk_b[h,ex,e_b,t]=np.exp(V_blue[h,ex,e_b,t])/(np.exp(V_school[h,ex,e_b,t])+np.exp(V_work[h,ex,e_b,t])+np.exp(V_home[h,ex,e_b,t])+np.exp(V_blue[h,ex,e_b,t])) # computer choice probabilities som er closed form når det er logit.
    
#print(pk)
#print(hk)
print(pk_w)
print(pk_s)
print(pk_b)
#print(V_school)
#print(V_work)
#print(V_home)
print(ev0) 

#Næste steps:
#Gennemgå koden. Se om den er okay/rigtig. 
#Mangler nok stadig noget stokasticitet i form af expected max i nuværende periode (mangler logsum)
#Se nfxp forelæsning omkring slide 16.
#Så problemet er, at jeg ikke tager højde for i udregningen af ev0, at der også er fejlled og at man 
#kan tage dem og dermed de bedste trækninger med i nytteberegningen. 
#Så logsum er en konsekvens af EV1 præferenceshocks. --> Smooth/integrated value function.
#Derudover kan der være specifikke lønshocks som stadig skal integreres.
#Simulér data for den store model. 
#Estimér modellen. 


[[[[0.1195562  0.11676445 0.11470894 0.11544539 0.        ]
   [0.11667478 0.11290373 0.10946135 0.10794544 0.        ]]

  [[0.14926031 0.15502515 0.15760026 0.1562648  0.        ]
   [0.14642708 0.15057929 0.15079174 0.14655236 0.        ]]

  [[0.15631908 0.16903866 0.18291296 0.19803641 0.        ]
   [0.15371163 0.16492956 0.17620776 0.18697045 0.        ]]]


 [[[0.19281861 0.18926558 0.18179206 0.17180068 0.        ]
   [0.18829198 0.18310008 0.17356246 0.16066526 0.        ]]

  [[0.2159097  0.22648398 0.2367405  0.24344693 0.        ]
   [0.21181411 0.22059633 0.22791653 0.23039214 0.        ]]

  [[0.21716686 0.22967102 0.24445057 0.26118259 1.        ]
   [0.21312645 0.22392961 0.23591034 0.24819607 1.        ]]]


 [[[0.30513969 0.30255491 0.29598564 0.28436352 1.        ]
   [0.29819159 0.29332346 0.28401032 0.26974619 1.        ]]

  [[0.32095605 0.33055281 0.34349318 0.36056357 1.        ]
   [0.31444683 0.32193227 0.3317374  0.34391165 1.        ]]

  [[0.32095605 0.330

In [24]:
#Nu skal jeg så forsøge at få et esktra valg og ekstra state ind. work_blue collar og e_b. e_bt+1=e_b+dt(e_bt)

#Nu er blue collar et bread and butter job. Der er returns to HK men du får ikke erfairng.

#Angiv parameterværdier. 
SU=0.8 # Det ændrer hvad man vælger
beta=1.0 #Discount factor
B=0.5 #Coefficient til humankapital
C=0.35 #Coefficient til erfaring
K_b=1 #Constant til blue collar work
B_b=0.1 #Coefficient til HK, blue collar
H=1.0 #Coefficient til at være hjemme.
T=3 #Antal tidsperioder
max_HK=3 #Maksimalt niveau af uddannelse.
max_e=T #Maksimalt niveau af erfaring. Samme som tidsperioder.

#Generer min expected value function. Nu skal den have en ekstra dimension.

ev0 = np.nan+np.zeros([max_HK,max_e,T]) #Generer expected value function med ekstra dimension (første er hk, anden er erfaring, sidste er tid)
pk_w = np.nan+np.zeros([max_HK,max_e,T]) # Choice prob for choosing work
pk_b = np.nan+np.zeros([max_HK,max_e,T]) #Choice prob for choosing blue collar
pk_s = np.nan+np.zeros([max_HK,max_e,T]) # Choice prob for choosing school
V_school=np.nan+np.zeros([max_HK,max_e,T]) # Choice specific value functions
V_work=np.nan+np.zeros([max_HK,max_e,T]) # Choice specific value functions
V_home=np.nan+np.zeros([max_HK,max_e,T]) # Choice specific value functions
V_blue=np.nan+np.zeros([max_HK,max_e,T]) # Choice specific value functions

#Nu genererer jeg expected value i last period for hver af de enkelte valg:
V_school[:,:,T-1]=SU # Last period value from schooling
#print(V_school)
for e in range(max_e):
    V_work[:,e,T-1]=B*np.arange(max_HK)+C*e

for e in range(max_e):
    V_blue[:,e,T-1]=K_b+B_b*np.arange(max_HK) +0*e
#print(V_work)
V_home[:,:,T-1]=H
#print(V_home)
ev0[:,:,T-1]=np.maximum.reduce([V_work[:,:,T-1],V_school[:,:,T-1],V_home[:,:,T-1],V_blue[:,:,T-1]])


pk_w[:,:,T-1]=ev0[:,:,T-1]==V_work[:,:,T-1] #Uden stokastik
#pk_w[:,:,T-1]=np.exp(V_work[:,:,T-1])/(np.exp(V_school[:,:,T-1])+np.exp(V_work[:,:,T-1])+np.exp(V_home[:,:,T-1])+np.exp(V_blue[:,:,T-1])) # MED EV1 error terms er det logit form expression.
pk_s[:,:,T-1]=ev0[:,:,T-1]==V_school[:,:,T-1] #Uden stokastik
#pk_s[:,:,T-1]=np.exp(V_school[:,:,T-1])/(np.exp(V_school[:,:,T-1])+np.exp(V_work[:,:,T-1])+np.exp(V_home[:,:,T-1])+np.exp(V_blue[:,:,T-1])) # MED EV1 error terms er det logit form expression.
pk_b[:,:,T-1]=ev0[:,:,T-1]==V_blue[:,:,T-1] #Uden stokastik
#pk_b[:,:,T-1]=np.exp(V_blue[:,:,T-1])/(np.exp(V_school[:,:,T-1])+np.exp(V_work[:,:,T-1])+np.exp(V_home[:,:,T-1])+np.exp(V_blue[:,:,T-1])) # MED EV1 error terms er det logit form expression.


for t in range(T-2, -1, -1): #Solve backwards from period T-1(2??), until period 0, backwards
    for h in range(max_HK):
        for ex in range(max_e):
            V_school[h,ex,t]=SU + beta*ev0[np.minimum(h+1,max_HK-1),ex,t+1] #Denne viser værdien af skole som er, værdien af SU samt den forventede value function hvor vi opdaterer hk., i t+1
            V_work[h,ex,t]= B*h + C*ex + beta*ev0[h,np.minimum(ex+1,max_e-1),t+1] #Denne viser værdien af work som er lig state space matricen evalueret der hvor vi er + den forventede value function i periode t+1
            V_home[h,ex,t]=H + beta*ev0[h,ex,t+1] #Denne viser værdien af home.
            V_blue[h,ex,t]=K_b+B_b*h+beta*ev0[h,ex,t+1] # Værdien af blue collar work
            ev0[h,ex,t]=np.maximum.reduce([V_work[h,ex,t],V_school[h,ex,t],V_home[h,ex,t],V_blue[h,ex,t]]) #Denne viser expected value function som er lig den maksimale værdi vi får fra den chocie specific value fct.
            pk_w[h,ex,t]=ev0[h,ex,t]==V_work[h,ex,t] #Uden stokastik
            pk_s[h,ex,t]=ev0[h,ex,t]==V_school[h,ex,t]
            pk_b[h,ex,t]=ev0[h,ex,t]==V_blue[h,ex,t]
            #pk_w[h,ex,t]=np.exp(V_work[h,ex,t])/(np.exp(V_school[h,ex,t])+np.exp(V_work[h,ex,t])+np.exp(V_home[h,ex,t])+np.exp(V_blue[h,ex,t])) # computer choice probabilities som er closed form når det er logit.
            #pk_s[h,ex,t]=np.exp(V_school[h,ex,t])/(np.exp(V_school[h,ex,t])+np.exp(V_work[h,ex,t])+np.exp(V_home[h,ex,t])+np.exp(V_blue[h,ex,t])) # computer choice probabilities som er closed form når det er logit.
            #pk_s[h,ex,t]=np.exp(V_blue[h,ex,t])/(np.exp(V_school[h,ex,t])+np.exp(V_work[h,ex,t])+np.exp(V_home[h,ex,t])+np.exp(V_blue[h,ex,t])) # computer choice probabilities som er closed form når det er logit.
    
    #print(pk)

#print(hk)
print(pk_w)
print(pk_s)
print(pk_b)
print(V_school)
print(V_work)
print(V_home)
print(ev0) 


[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 1.]]

 [[1. 0. 0.]
  [1. 1. 1.]
  [1. 1. 1.]]]
[[[1. 0. 0.]
  [1. 0. 0.]
  [1. 1. 0.]]

 [[0. 0. 0.]
  [1. 0. 0.]
  [1. 1. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]
[[[1. 1. 1.]
  [1. 1. 1.]
  [0. 1. 1.]]

 [[1. 1. 1.]
  [0. 1. 1.]
  [0. 0. 0.]]

 [[0. 1. 1.]
  [0. 0. 0.]
  [0. 0. 0.]]]
[[[3.   1.9  0.8 ]
  [3.   1.9  0.8 ]
  [3.3  2.   0.8 ]]

 [[3.2  2.   0.8 ]
  [3.85 2.15 0.8 ]
  [4.2  2.5  0.8 ]]

 [[3.2  2.   0.8 ]
  [3.85 2.15 0.8 ]
  [4.2  2.5  0.8 ]]]
[[[2.   1.   0.  ]
  [2.35 1.35 0.35]
  [2.7  1.7  0.7 ]]

 [[2.7  1.6  0.5 ]
  [3.35 2.05 0.85]
  [3.7  2.4  1.2 ]]

 [[4.05 2.35 1.  ]
  [4.75 3.05 1.35]
  [5.1  3.4  1.7 ]]]
[[[3.   2.   1.  ]
  [3.   2.   1.  ]
  [3.   2.   1.  ]]

 [[3.2  2.1  1.  ]
  [3.2  2.1  1.  ]
  [3.5  2.2  1.  ]]

 [[3.4  2.2  1.  ]
  [4.05 2.35 1.  ]
  [4.4  2.7  1.  ]]]
[[[3.   2.   1.  ]
  [3.   2.   1.  ]
  [3.3  2.   1.  ]]

 [[3.3  2.2  1.1 ]
  [3.85 2.2  1.1 ]
  [

In [74]:
#Her forsøger jeg udover en anden state OGSÅ at indføre et tredje alternativ, som er stay home.

#Angiv parameterværdier. 
SU=0.8 # Det ændrer hvad man vælger
beta=1.0 #Discount factor
B=0.5 #Coefficient til humankapital
C=5.35 #Coefficient til erfaring
H=1 #Coefficient til at være hjemme.
T=3 #Antal tidsperioder
max_HK=2 #Maksimalt niveau af uddannelse.
max_e=T #Maksimalt niveau af erfaring. Samme som tidsperioder.

#Generer min expected value function. Nu skal den have en ekstra dimension.

ev0 = np.nan+np.zeros([max_HK,max_e,T]) #Generer expected value function med ekstra dimension (første er hk, anden er erfaring, sidste er tid)
pk_w = np.nan+np.zeros([max_HK,max_e,T]) # Choice prob for choosing work
pk_s = np.nan+np.zeros([max_HK,max_e,T]) # Choice prob for choosing school
V_school=np.nan+np.zeros([max_HK,max_e,T]) # Choice specific value functions
V_work=np.nan+np.zeros([max_HK,max_e,T]) # Choice specific value functions
V_home=np.nan+np.zeros([max_HK,max_e,T]) # Choice specific value functions

#Nu genererer jeg expected value i last period for hver af de enkelte valg:
V_school[:,:,T-1]=SU # Last period value from schooling
#print(V_school)
for e in range(max_e):
    V_work[:,e,T-1]=B*np.arange(max_HK)+C*e

#print(V_work)
V_home[:,:,T-1]=H
#print(V_home)
print(type(V_home))
ev0[:,:,T-1]=np.maximum.reduce([V_work[:,:,T-1],V_school[:,:,T-1],V_home[:,:,T-1]])
#print(V_work) # Okay så matricen afgør antal enheder af HK, rækken afgør enheder af erfaring. Kolonnen afgør tid. 
#print(ev0) ##check

pk_w[:,:,T-1]=ev0[:,:,T-1]==V_work[:,:,T-1] #Uden stokastik
#pk_w[:,:,T-1]=np.exp(V_work[:,:,T-1])/(np.exp(V_school[:,:,T-1])+np.exp(V_work[:,:,T-1])+np.exp(V_home[:,:,T-1])) # MED EV1 error terms er det logit form expression.
pk_s[:,:,T-1]=ev0[:,:,T-1]==V_school[:,:,T-1] #Uden stokastik
#pk_s[:,:,T-1]=np.exp(V_school[:,:,T-1])/(np.exp(V_school[:,:,T-1])+np.exp(V_work[:,:,T-1])+np.exp(V_home[:,:,T-1])) # MED EV1 error terms er det logit form expression.
#print(pk)
#print(pk_w)
#print(pk_s)

#Nu kører jeg med loopet:
for t in range(T-2, -1, -1): #Solve backwards from period T-1(2??), until period 0, backwards
    for h in range(max_HK):
        for ex in range(max_e):
            V_school[h,ex,t]=SU + beta*ev0[np.minimum(h+1,max_HK-1),ex,t+1] #Denne viser værdien af skole som er, værdien af SU samt den forventede value function hvor vi opdaterer hk., i t+1
            V_work[h,ex,t]= B*h + C*ex + beta*ev0[h,np.minimum(ex+1,max_e-1),t+1] #Denne viser værdien af work som er lig state space matricen evalueret der hvor vi er + den forventede value function i periode t+1
            V_home[h,ex,t]=H + beta*ev0[h,ex,t+1] #Denne viser værdien af home.
            ev0[h,ex,t]=np.maximum.reduce([V_work[h,ex,t],V_school[h,ex,t],V_home[h,ex,t]]) #Denne viser expected value function som er lig den maksimale værdi vi får fra den chocie specific value fct.
            pk_w[h,ex,t]=ev0[h,ex,t]==V_work[h,ex,t] #Uden stokastik
            pk_s[h,ex,t]=ev0[h,ex,t]==V_school[h,ex,t]
            #pk_w[h,ex,t]=np.exp(V_work[h,ex,t])/(np.exp(V_school[h,ex,t])+np.exp(V_work[h,ex,t])+np.exp(V_home[h,ex,t])) # computer choice probabilities som er closed form når det er logit.
            #pk_s[h,ex,t]=np.exp(V_school[h,ex,t])/(np.exp(V_school[h,ex,t])+np.exp(V_work[h,ex,t])+np.exp(V_home[h,ex,t])) # computer choice probabilities som er closed form når det er logit.
        
    #print(pk)

#print(hk)
print(pk_w)
print(pk_s)
print(V_school)
print(V_work)
print(V_home)
print(ev0) 


<class 'numpy.ndarray'>
[[[1. 1. 0.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 0.]
  [1. 1. 1.]
  [1. 1. 1.]]]
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]
[[[ 7.15  1.8   0.8 ]
  [17.85  6.65  0.8 ]
  [23.2  12.    0.8 ]]

 [[ 7.15  1.8   0.8 ]
  [17.85  6.65  0.8 ]
  [23.2  12.    0.8 ]]]
[[[16.05  5.35  0.  ]
  [26.75 16.05  5.35]
  [32.1  21.4  10.7 ]]

 [[17.55  6.35  0.5 ]
  [28.25 17.05  5.85]
  [33.6  22.4  11.2 ]]]
[[[ 6.35  2.    1.  ]
  [17.05  6.35  1.  ]
  [22.4  11.7   1.  ]]

 [[ 7.35  2.    1.  ]
  [18.05  6.85  1.  ]
  [23.4  12.2   1.  ]]]
[[[16.05  5.35  1.  ]
  [26.75 16.05  5.35]
  [32.1  21.4  10.7 ]]

 [[17.55  6.35  1.  ]
  [28.25 17.05  5.85]
  [33.6  22.4  11.2 ]]]


In [70]:
#Her forsøger jeg så at introducere en anden state, som er erfaring og som akkumuleres ved at arbejde. Udvikling: et+1 = et + dt

#Angiv parameterværdier. 
SU=0.8 # Det ændrer hvad man vælger
beta=1.0 #Discount factor
B=0.5 #Coefficient til humankapital
C=0.35 #Coefficient til erfaring
T=4 #Antal tidsperioder
max_HK=3 #Maksimalt niveau af uddannelse.
max_e=2 #Maksimalt niveau af erfaring. Samme som tidsperioder. 

#Generer min expected value function. Nu skal den have en ekstra dimension.

ev0 = np.nan+np.zeros([max_HK,max_e,T]) #Generer expected value function med ekstra dimension (første er hk, anden er erfaring, sidste er tid)
pk = np.nan+np.zeros([max_HK,max_e,T]) # Choice prob for choosing work
V_school=np.nan+np.zeros([max_HK,max_e,T]) # Choice specific value functions
V_work=np.nan+np.zeros([max_HK,max_e,T]) # Choice specific value functions

#Nu genererer jeg expected value i last period for hver af de enkelte valg:
V_school[:,:,T-1]=SU # Last period value from schooling
#print(V_school)
for e in range(max_e):
    V_work[:,e,T-1]=B*np.arange(max_HK)+C*e
#print(V_work) # Okay så matricen afgør antal enheder af HK, rækken afgør enheder af erfaring. Kolonnen afgør tid. 
ev0[:,:,T-1]=np.maximum(V_school[:,:,T-1],V_work[:,:,T-1]) 
#print(ev0)


#Nu genererer jeg ccp. Både med og uden stokastik:
#Now I generate the choice probabilities in the last period 
pk[:,:,T-1]=ev0[:,:,T-1]==V_work[:,:,T-1] #Uden stokastik
#pk[:,:,T-1]=np.exp(V_work[:,:,T-1])/(np.exp(V_school[:,:,T-1])+np.exp(V_work[:,:,T-1])) # MED EV1 error terms er det logit form expression.
#print(pk) # OK pointen er, at jo mere HK og erfaring du har, jo mere sandsynligt er det, at du arbejder sidste periode. 

#Hmm det ser fair nok ud

#Nu prøver jeg så, at køre loops:

for t in range(T-2, -1, -1): #Solve backwards from period T-1(2??), until period 0, backwards
    for h in range(max_HK):
        for ex in range(max_e):
            V_school[h,ex,t]=SU + beta*ev0[np.minimum(h+1,max_HK-1),ex,t+1] #Denne viser værdien af skole som er, værdien af SU samt den forventede value function hvor vi opdaterer hk., i t+1
            V_work[h,ex,t]= B*h + C*ex + beta*ev0[h,np.minimum(ex+1,max_e-1),t+1] #Denne viser værdien af work som er lig state space matricen evalueret der hvor vi er + den forventede value function i periode t+1
            ev0[h,ex,t]=np.maximum(V_work[h,ex,t],V_school[h,ex,t]) #Denne viser expected value function som er lig den maksimale værdi vi får fra den chocie specific value fct.
            pk[h,ex,t]=ev0[h,ex,t]==V_work[h,ex,t] #Uden stokastik
            #pk[h,ex,t]=np.exp(V_work[h,ex,t])/(np.exp(V_school[h,ex,t])+np.exp(V_work[h,ex,t])) # computer choice probabilities som er closed form når det er logit.
        #print(pk)

#print(hk)
print(pk)
#print(V_school)
#print(V_work)
#print(ev0)  


[[[0.26894142 0.27888482 0.31002552 0.31002552]
  [0.26894142 0.27888482 0.37754067 0.38936077]]

 [[0.37754067 0.37754067 0.38936077 0.42555748]
  [0.37754067 0.37754067 0.38936077 0.5124974 ]]

 [[0.63413559 0.63413559 0.63413559 0.549834  ]
  [0.63413559 0.63413559 0.63413559 0.63413559]]]


In [156]:

ccp=np.empty(T)*0
#print(ccp)
start_HK=0
start_e=0

#Måske er det i virkeligheden her, jeg skal have transition på.
#Skal måske faktisk ikke bruges. Måske skal der bare være en der 
# Second last period:



#Nu skal jeg gøre sådan så man opdaterer humankapital. Så vi gør sådan, 

#Se om jeg kan få det samme bare med transition matrix.
#Tran_w=np.eye(T)
#print(Tran_w)
#Tran_s=np.zeros([max_HK,T])
#Tran_s[0,1]=1
#Tran_s[1,2]=1
#Tran_s[2,2]=1
#print(Tran_s)

for t in range(T):
    ccp[t]=pk[start_HK,start_e,t]
    if ev0[start_HK,start_e,t]==V_school[start_HK,start_e,t]:
        start_HK=start_HK+1
    elif ev0[start_HK,start_e,t]==V_school[start_HK,start_e,t]:
        start_e=start_e+1
    else:
        start=start
print(ccp)


[0. 0. 1.]


In [107]:
# Løser modellen via backwards induction.
#Denne er den simple med to states. Som virker. 

#Angiv parameterværdier. 
SU=0.8 # Det ændrer hvad man vælger
beta=1.0 #Discount factor
B=0.5 #Coefficient til humankapital
T=3 #Antal tidsperioder
max_HK=3 #Maksimalt niveau af uddannelse.


#Initialize:
hk = np.nan+np.zeros([max_HK,T]) #Genererer state space matrix. 
for t in range(T):
     # State variables
    hk[:,T-(t+1)] = np.arange(max_HK) #Human capital er den eneste state variable pt. 
#print(hk)

ev0 = np.nan+np.zeros([max_HK,T]) #Generer expected value function.
pk = np.nan+np.zeros([max_HK,T]) # Choice prob for choosing work
V_school=np.nan+np.zeros([max_HK,T]) # Choice specific value functions
V_work=np.nan+np.zeros([max_HK,T]) # Choice specific value functions


#Now I generate the expected value for THE LAST PERIOD from both alternatives and the value function (which is the max of the two)
V_school[:,T-1]=SU # Last period value from schooling
V_work[:,T-1]=B*np.arange(max_HK) # Last period value from work depending on HK
ev0[:,T-1]=np.maximum(V_school[:,T-1],V_work[:,T-1]) 

#Now I generate the choice probabilities in the last period 
pk[:,T-1]=ev0[:,T-1]==V_work[:,T-1] #Uden stokastik
#pk[:,T-1]=np.exp(V_work[:,T-1])/(np.exp(V_school[:,T-1])+np.exp(V_work[:,T-1])) # MED EV1 error terms er det logit form expression. 


#Her løser jeg modellen via backwards induction for periode T-1 indtil for periode 0
for t in range(T-2, -1, -1): #Solve backwards from period T-1(2??), until period 0, backwards
    for h in range(max_HK):
        V_school[h,t]=SU + beta*ev0[np.minimum(h+1,max_HK-1),t+1] #Denne viser værdien af skole som er, værdien af SU samt den forventede value function hvor vi opdaterer hk., i t+1
        V_work[h,t]= B*h + beta*ev0[h,t+1] #Denne viser værdien af work som er lig state space matricen evalueret der hvor vi er + den forventede value function i periode t+1
        ev0[h,t]=np.maximum(V_work[h,t],V_school[h,t]) #Denne viser expected value function som er lig den maksimale værdi vi får fra den chocie specific value fct.
        pk[h,t]=ev0[h,t]==V_work[h,t] #Uden stokastik
        #pk[h,t]=np.exp(V_work[h,t])/(np.exp(V_school[h,t])+np.exp(V_work[h,t])) # computer choice probabilities som er closed form når det er logit.
        #print(pk)

#print(hk)
print(pk)
print(V_school)
print(V_work)
print(ev0)  


#Næste steps:
#Få et ekstra valg på. Dette kan for eksempel være at man skal være derhjemme. For at gøre det skal vi have en ekstra nyttefunktion samt en ekstra choice prob. 
#Få en ekstra state på. Dette kan for eksempel være arbejdserfaring. For at gøre det skal vi have en ekstra dimension i vores state space og øvrige matricer. 
#Forstå helt hvad det er der sker i zurcher-modellen samt i cake eating og hvordan det relaterer sig til career decision modellen.
#Simuler noget data og forsøg at estimér parametrene i modellen ved hjælp af NFXP. Kræver dog at jeg kender likelihood funktionen.
#Lav counterfactuals hvor jeg ændrer SU-raten. 
#Overvej om jeg skal have state transitions på på en måde. 
#Estimer den på registerdata eller ved hjælp af moments. 

[[0. 0. 0.]
 [0. 0. 0.]
 [1. 1. 1.]]
[[2.6 1.6 0.8]
 [2.8 1.8 0.8]
 [2.8 1.8 0.8]]
[[1.6 0.8 0. ]
 [2.3 1.3 0.5]
 [3.  2.  1. ]]
[[2.6 1.6 0.8]
 [2.8 1.8 0.8]
 [3.  2.  1. ]]


In [26]:
# Løser modellen via backwards induction.
#VIRKER!!!

#Angiv parameterværdier. 
SU=0.8 # Det ændrer hvad man vælger
beta=1.0 #Discount factor
B=0.5 #Coefficient til humankapital
T=3 #Antal tidsperioder
max_HK=3 #Maksimalt niveau af uddannelse.


#Initialize:
hk = np.nan+np.zeros([max_HK,T]) #Genererer state space matrix. 
for t in range(T):
     # State variables
    hk[:,T-(t+1)] = np.arange(max_HK) #Human capital er den eneste state variable pt. 
#print(hk)

ev0 = np.nan+np.zeros([max_HK,T]) #Generer expected value function.
pk = np.nan+np.zeros([max_HK,T]) # Choice prob for choosing work
V_school=np.nan+np.zeros([max_HK,T]) # Choice specific value functions
V_work=np.nan+np.zeros([max_HK,T]) # Choice specific value functions


#Now I generate the expected value for THE LAST PERIOD from both alternatives and the value function (which is the max of the two)
V_school[:,T-1]=SU # Last period value from schooling
V_work[:,T-1]=B*hk[:,T-1] # Last period value from work depending on HK
ev0[:,T-1]=np.maximum(V_school[:,T-1],V_work[:,T-1]) 

#Now I generate the choice probabilities in the last period 
pk[:,T-1]=ev0[:,T-1]==V_work[:,T-1] #Uden stokastik
#pk[:,T-1]=np.exp(V_work[:,T-1])/(np.exp(V_school[:,T-1])+np.exp(V_work[:,T-1])) # MED EV1 error terms er det logit form expression. 


#Her løser jeg modellen via backwards induction for periode T-1 indtil for periode 0
for t in range(T-2, -1, -1): #Solve backwards from period T-1(2??), until period 0, backwards
    for h in range(max_HK):
        V_school[h,t]=SU + beta*ev0[np.minimum(h+1,max_HK-1),t+1] #Denne viser værdien af skole som er, værdien af SU samt den forventede value function hvor vi opdaterer hk., i t+1
        V_work[h,t]= B*hk[h,t] + beta*ev0[h,t+1] #Denne viser værdien af work som er lig state space matricen evalueret der hvor vi er + den forventede value function i periode t+1
        ev0[h,t]=np.maximum(V_work[h,t],V_school[h,t]) #Denne viser expected value function som er lig den maksimale værdi vi får fra den chocie specific value fct.
        pk[h,t]=ev0[h,t]==V_work[h,t] #Uden stokastik
        #pk[h,t]=np.exp(V_work[h,t])/(np.exp(V_school[h,t])+np.exp(V_work[h,t])) # computer choice probabilities som er closed form når det er logit.
        #print(pk)

#print(hk)
print(pk)
print(V_school)
print(V_work)
print(ev0)  


#Næste steps:
#Få et ekstra valg på. Dette kan for eksempel være at man skal være derhjemme. For at gøre det skal vi have en ekstra nyttefunktion samt en ekstra choice prob. 
#Få en ekstra state på. Dette kan for eksempel være arbejdserfaring. For at gøre det skal vi have en ekstra dimension i vores state space og øvrige matricer. 
#Forstå helt hvad det er der sker i zurcher-modellen samt i cake eating og hvordan det relaterer sig til career decision modellen.
#Simuler noget data og forsøg at estimér parametrene i modellen ved hjælp af NFXP. Kræver dog at jeg kender likelihood funktionen.
#Lav counterfactuals hvor jeg ændrer SU-raten. 
#Overvej om jeg skal have state transitions på på en måde. 
#Estimer den på registerdata eller ved hjælp af moments. 



[[0. 0. 0.]
 [0. 0. 0.]
 [1. 1. 1.]]
[[2.6 1.6 0.8]
 [2.8 1.8 0.8]
 [2.8 1.8 0.8]]
[[1.6 0.8 0. ]
 [2.3 1.3 0.5]
 [3.  2.  1. ]]
[[2.6 1.6 0.8]
 [2.8 1.8 0.8]
 [3.  2.  1. ]]


In [27]:
# Simuler modellen

import pandas as pd

idx = np.tile(np.arange(1,N+1),(T,1))  # Index
t = np.tile(np.arange(1,T+1),(N,1)).T # Tidsperiode for hvert individ.
N=10 #Amount of people to simulate. 
x=np.zeros([T,N]) # states for all the simulated people in all the simulated time. 
x1=np.zeros([T,N]) # states for all the simulated people in all the simulated time. 
d=np.nan + np.zeros((T,N)) # decisions for all people. 1 = work, 0 = school. 
u_init=0
u_d=np.random.rand(T,N) # Den der gør om man vælger den ene eller den anden.
#pks=np.zeros([max_HK,N,T])
#print(pks)
x[0,:]=u_init # First state variable. 


#print(pk[int(x[1,1]),1])

for n in range(N):
    for it in range(T):
        d[it,n] = u_d[it,n]<pk[int(x[it,n]),it] # d=1=work. jo mindre u bliver jo mere ssh for work. Så vel i virkeligheden 
        x1[it,n] = np.minimum(x[it,n]+(1-d[it,n]),max_HK-1)
        if it < T-1:
            x[it+1,n] = x1[it,n]    

idx =  np.reshape(idx,T*N,order='F')
t = np.reshape(t,T*N,order='F')
d = np.reshape(d,T*N,order='F')
x = np.reshape(x,T*N,order='F')
x1 = np.reshape(x1,T*N,order='F')
#dx1 = np.reshape(dx1,T*N,order='F')


data = {'id': idx,'t': t, 'd': d, 'x': x, 'x1': x1}
df= pd.DataFrame(data)
    
df

NameError: name 'N' is not defined

In [ ]:

ccp=np.empty(T)*0
#print(ccp)
start=0

#Måske er det i virkeligheden her, jeg skal have transition på.
#Skal måske faktisk ikke bruges. Måske skal der bare være en der 
# Second last period:



#Nu skal jeg gøre sådan så man opdaterer humankapital. Så vi gør sådan, 

#Se om jeg kan få det samme bare med transition matrix.
#Tran_w=np.eye(T)
#print(Tran_w)
#Tran_s=np.zeros([max_HK,T])
#Tran_s[0,1]=1
#Tran_s[1,2]=1
#Tran_s[2,2]=1
#print(Tran_s)

for t in range(T):
    ccp[t]=pk[start,t]
    if ev0[start,t]==V_school[start,t]:
        start=start+1
    else:
        start=start
print(ccp)


In [ ]:
       
        
    def sim_data(self,N,T,pk): 

        np.random.seed(2020)
        
        # Index 
        idx = np.tile(np.arange(1,N+1),(T,1))  
        t = np.tile(np.arange(1,T+1),(N,1)).T
        
        # Draw random numbers
        u_init = np.random.randint(self.n,size=(1,N))
        u_dx = np.random.rand(T,N)
        u_d = np.random.rand(T,N)
        
        # Find states and choices
        csum_p = np.cumsum(self.p)
        dx1 = 0
        for val in csum_p:
            dx1 += u_dx>val
        
        x = np.zeros((T,N),dtype=int)
        x1 =  np.zeros((T,N),dtype=int)
        d = np.nan + np.zeros((T,N))
        x[0,:] = u_init # initial condition
        for it in range(T):
            d[it,:] = u_d[it,:]<1-pk[x[it,:]]  # replace = 1 , keep = 0   
            x1[it,:] = np.minimum(x[it,:]*(1-d[it,:]) + dx1[it,:] , self.n-1)
            if it < T-1:
                x[it+1,:] = x1[it,:]
                
        
        # reshape 
        idx =  np.reshape(idx,T*N,order='F')
        t = np.reshape(t,T*N,order='F')
        d = np.reshape(d,T*N,order='F')
        x = np.reshape(x,T*N,order='F')
        x1 = np.reshape(x1,T*N,order='F')
        dx1 = np.reshape(dx1,T*N,order='F')


        data = {'id': idx,'t': t, 'd': d, 'x': x, 'x1': x1, 'dx1': dx1}
        df= pd.DataFrame(data) 

        return df

In [40]:
# Denne kode virker! Både for stokastik og ikke stokastik.



SU=0.8 # Det ændrer hvad man vælger
beta=1
B=0.5
T=3
max_HK=3

 # 

ev0 = np.nan+np.zeros([max_HK,T]) # Value function

hk = np.nan+np.zeros([max_HK,T])
for t in range(T):
     # State variables
    hk[:,T-(t+1)] = np.arange(max_HK) #Human capital
#print(hk)


pk = np.nan+np.zeros([max_HK,T]) # Choice prob for choosing work
V_school=np.nan+np.zeros([max_HK,T]) # Choice specific value functions
V_work=np.nan+np.zeros([max_HK,T]) # Choice specific value functions
maxV=np.nan+np.zeros([max_HK,T])
logsum=np.nan+np.zeros([max_HK,T])

V_school[:,T-1]=SU # Last period value from schooling
V_work[:,T-1]=B*hk[:,T-1] # Last period value from work depending on HK

maxV[:,T-1]=np.maximum(V_school[:,T-1],V_work[:,T-1]) 
logsum[:,T-1]=maxV[:,T-1]
ev0[:,T-1]=logsum[:,T-1] #Value function for last period
pk[:,T-1]=ev0[:,T-1]==V_work[:,T-1] #Uden stokastik
#pk[:,T-1]=1/(1+np.exp(V_school[:,T-1]-V_work[:,T-1])) # Med stokastik

# Second last period:



#Nu skal jeg gøre sådan så man opdaterer humankapital. Så vi gør sådan, 

#Se om jeg kan få det samme bare med transition matrix.

    
for t in range(T-2, -1, -1): #Solve backwards from period T-2, until period 0, backwards
    for h in range(max_HK):
        V_school[h,t]=SU + beta*ev0[np.minimum(h+1,max_HK-1),t+1] #Hvor der så kommer en ekstra HK på.
        V_work[h,t]= B*hk[h,t+1] + beta*ev0[h,t+1]
        ev0[h,t]=np.maximum(V_work[h,t],V_school[h,t])
        #logsum[h,t]=maxV[h,t]
        #ev0[h,t]=logsum[h,t]
        pk[h,t]=ev0[h,t]==V_work[h,t] #Uden stokastik
        #hk_plus1=1 if t-1=school
        #pk[h,t]=1/(1+np.exp(V_school[h,t]-V_work[h,t])) # Med stokastik
        #print(pk)

#print(hk)
print(pk)
print(V_school)
print(V_work)
print(ev0) 


#Nu skal jeg så have en simulering af hvad en person vælger i de tre perioder:
#Det er noget med at køre et loop for hver periode også sætte ind i ev0 osv. ovenfor.




#pk[1,1]=1/(1+np.exp(V_school[1,1]-V_work[1,1])
#pk


#Overgangssandsynligheder for at gå i skole. 

#Nu skal jeg lave en funktion, der viser conditional choice probability:
#Lav choice probabilities og benyt logsum. Jeg tror det er vejen frem.


#Så vi skal jo 

#Hvad har jeg gjort: 
#Jeg har lavet en model uden stokastistet, der finder den optimale løsning ved backwards induction.

#Hvad skal jeg nu:
#Nu vil jeg så lave den funktion jeg har lavet nedenfor, men bare mere generalizable. 
#Det vil sige, at den viser hvilket valg en person træffer i hver periode. 
#Herefter skal jeg have stokatiske stød med i modellen.
#Herefter skal jeg så simulere modellen for flere mennesker så man kan se hvilke valg folk træffer.
#Herefter kører jeg den så for forskellige SU-rater og finder effekten af SU.
#Herefter skal jeg udvide modellen så der kommer flere valg.
#Herefter skal jeg udvide modellen så der kommer flere states. 


[[0. 0. 0.]
 [0. 0. 0.]
 [1. 1. 1.]]
[[2.6 1.6 0.8]
 [2.8 1.8 0.8]
 [2.8 1.8 0.8]]
[[1.6 0.8 0. ]
 [2.3 1.3 0.5]
 [3.  2.  1. ]]
[[2.6 1.6 0.8]
 [2.8 1.8 0.8]
 [3.  2.  1. ]]


In [151]:
choices=np.nan+np.zeros(T)

#For first period:
if ev0[0,0]==V_work[0,0]:
    choices[0]=1
else:
    choices[0]=0

#For second period:
if choices[0]==0:
    if ev0[1,1]==V_work[1,1]:
        choices[1]=1
    else:
        choices[1]=0
else:
    if ev0[0,1]==V_work[0,1]:
        choices[1]=1
    else:
        choices[1]=0

#For third period:
if choices[1]==0 and choices[0]==0:
    if ev0[2,2]==V_work[2,2]:
        choices[2]=1
    else:
        choices[2]=0
elif choices[1]==0 or choices[0]==0:
    if ev0[1,2]==V_work[1,2]:
        choices[2]=1
    else:
        choices[2]=0
else:
    if ev0[0,2]==V_work[0,2]:
        choices[2]=1
    else:
        choices[2]=0

print(choices)

[0. 0. 1.]


In [ ]:
# 4. simulate
C_backwards = np.empty(T)
W_now = W

for t in range(T):
    W_now = int(W_now)   # change the type to integreger 0,1,2,3 and so on
    C_backwards[t] = C[W_now,t]  # Use the solution you find above 
    W_now = W_now-C_backwards[t]

print(C_backwards)

In [ ]:
def solve_backwards(beta,W,T):
        # 2. Initialize
    Vstar_bi = np.nan+np.zeros([W+1,T])
    Cstar_bi = np.nan + np.zeros([W+1,T])
    Cstar_bi[:,T-1] = np.arange(W+1) 
    Vstar_bi[:,T-1] = np.sqrt(Cstar_bi[:,T-1])
    # 3. solve
    
    # Loop over periods
    for t in range(T-2, -1, -1):  #from period T-2, until period 0, backwards  
        
        #loop over states
        for w in range(W+1):
            c = np.arange(w+1)
            w_c = w - c
            V_next = Vstar_bi[w_c,t+1]
            V_guess = np.sqrt(c)+beta*V_next
            Vstar_bi[w,t] = np.amax(V_guess)
            Cstar_bi[w,t] = np.argmax(V_guess)

    return Cstar_bi, Vstar_bi

In [72]:
# 1. Setup 
beta = 0.9
W = 5 # Det er human
T = 3

# 2. Initialize
Vstar_bi = np.nan+np.zeros([W+1,T])
print(Vstar_bi)
Cstar_bi = np.nan + np.zeros([W+1,T])
print(Cstar_bi)
Cstar_bi[:,T-1] = np.arange(W+1) 
print(Cstar_bi)
Vstar_bi[:,T-1] = np.sqrt(Cstar_bi[:,T-1])
print(Vstar_bi)
# 3. solve

# Loop over periods
for t in range(T-2, -1, -1):  #from period T-2, until period 0, backwards  
    
    #loop over states
    for w in range(W+1):
        c = np.arange(w+1)
        w_c = w - c
        V_next = Vstar_bi[w_c,t+1]
        V_guess = np.sqrt(c)+beta*V_next
        Vstar_bi[w,t] = np.amax(V_guess)
        Cstar_bi[w,t] = np.argmax(V_guess)
        
print(Cstar_bi)
print(Vstar_bi)
print(c)
print(w_c)

[[nan nan nan]
 [nan nan nan]
 [nan nan nan]
 [nan nan nan]
 [nan nan nan]
 [nan nan nan]]
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]
 [nan nan nan]
 [nan nan nan]
 [nan nan nan]]
[[nan nan  0.]
 [nan nan  1.]
 [nan nan  2.]
 [nan nan  3.]
 [nan nan  4.]
 [nan nan  5.]]
[[       nan        nan 0.        ]
 [       nan        nan 1.        ]
 [       nan        nan 1.41421356]
 [       nan        nan 1.73205081]
 [       nan        nan 2.        ]
 [       nan        nan 2.23606798]]
[[0. 0. 0.]
 [1. 1. 1.]
 [1. 1. 2.]
 [1. 2. 3.]
 [2. 2. 4.]
 [2. 3. 5.]]
[[0.         0.         0.        ]
 [1.         1.         1.        ]
 [1.9        1.9        1.41421356]
 [2.71       2.31421356 1.73205081]
 [3.12421356 2.68700577 2.        ]
 [3.49700577 3.00484301 2.23606798]]
[0 1 2 3 4 5]
[5 4 3 2 1 0]
